In [1]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU,Dropout2d,Dropout
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D,GRU_NCL

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds_log.npy')
means = np.load('../Data/means_log.npy')
long_normal_value = np.load('../Data/long_lognormal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = .5

In [5]:
# baseline model
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 106.31
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# np.mean(np.square(y - yhat))
# >>> 1.1114173

In [5]:
# set up data loader
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalized(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalized(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [25]:
conv = Sequential(ConvBatchRelu1D(3,8,3),
                  Dropout(0.1),
                  ConvBatchRelu1D(8,16,3),
                  Dropout(0.2),
                  ConvBatchRelu1D(16,32,3),
                  Dropout(0.3),
                  ConvBatchRelu1D(32,64,3),
                  Dropout(0.5),
                  ConvBatchRelu1D(64,128,3),
                  Dropout(0.6))

In [26]:
seq_model = conv

In [27]:
linear = Sequential(Linear(128,64),LeakyReLU(0.1,True),Dropout(0.5),Linear(64,1))

In [28]:
convert = Sequential(Linear(110,60),LeakyReLU(0.1,True),Dropout(0.5))

In [29]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized_NCL(seq_model,linear,convert).to('cuda:0')

In [30]:
loss_func_nor = loss_func_generator_normalized(MSELoss())
# loss_func_nor = loss_func_generator_normalized(L1Loss())
#loss_func_nor = loss_func_generator_normalized(SMAPE)

In [31]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [32]:
model_nor = fit(3, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip,patience=0)

epoch:0, train_loss:0.8068458331063622, val_loss:0.8406159281730652
epoch:1, train_loss:0.7737093387953358, val_loss:0.8701293468475342
epoch:2, train_loss:0.7637718478941546, val_loss:0.8746597170829773
Training completed in 49.20552587509155s


In [33]:
model_nor = fit(3, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip,patience=0,lossBest=0.8406159281730652)

epoch:0, train_loss:0.7736423231005051, val_loss:0.8738323450088501
epoch:1, train_loss:0.7646969530403305, val_loss:0.8833514451980591
epoch:2, train_loss:0.7604822883334185, val_loss:0.9003375172615051
Training completed in 49.13328671455383s


In [34]:
model = CNN_RNN2seq_logNormalized_NCL(seq_model,linear,convert).to('cuda:0')

In [35]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [36]:
loss_func = loss_func_generator(SMAPE)

In [37]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:38.062813891277415, val_loss:37.79411697387695
epoch:1, train_loss:37.83820589194026, val_loss:37.58927536010742
epoch:2, train_loss:37.733674874577495, val_loss:37.209144592285156
epoch:3, train_loss:37.72064845426095, val_loss:37.24424362182617
epoch:4, train_loss:37.66216705598979, val_loss:37.532955169677734
Training completed in 96.11725068092346s


In [38]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.72080066265838, val_loss:37.402931213378906
epoch:1, train_loss:37.599040530506194, val_loss:37.273502349853516
epoch:2, train_loss:37.55847692440211, val_loss:37.09085464477539
Training completed in 58.10332131385803s


In [39]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip,lossBest=37.09085464477539)

epoch:0, train_loss:37.622895582722876, val_loss:37.062557220458984
epoch:1, train_loss:37.500365000314666, val_loss:36.955169677734375
epoch:2, train_loss:37.534593821313095, val_loss:36.611995697021484
Training completed in 57.837735652923584s


In [40]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip,lossBest=36.611995697021484)

epoch:0, train_loss:37.49086432778156, val_loss:36.972137451171875
epoch:1, train_loss:37.42799619862453, val_loss:36.60200881958008
epoch:2, train_loss:37.384181678851036, val_loss:36.831993103027344
Training completed in 57.70332932472229s


In [41]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip,lossBest=36.602)

epoch:0, train_loss:37.453094677109796, val_loss:36.82497024536133
epoch:1, train_loss:37.38498774810159, val_loss:36.749813079833984
epoch:2, train_loss:37.39038259081272, val_loss:36.554588317871094
Training completed in 57.62628221511841s


In [42]:
model = fit(6, model, loss_func, opt, train_gen, val_gen,clip,lossBest=36.554588317871094)

epoch:0, train_loss:37.33007209461588, val_loss:36.761878967285156
epoch:1, train_loss:37.36564198923852, val_loss:36.53905487060547
epoch:2, train_loss:37.376305666118085, val_loss:36.71700668334961
epoch:3, train_loss:37.356879772917594, val_loss:36.68766784667969
epoch:4, train_loss:37.30729416071442, val_loss:36.281307220458984
epoch:5, train_loss:37.32709844520055, val_loss:36.490234375
Training completed in 115.63556814193726s


In [43]:
model = fit(10, model, loss_func, opt, train_gen, val_gen,clip,lossBest=36.281)

epoch:0, train_loss:37.320939521591896, val_loss:36.31989288330078
epoch:1, train_loss:37.23228143879787, val_loss:36.52996826171875
epoch:2, train_loss:37.30782368516675, val_loss:36.29140853881836
epoch:3, train_loss:37.281723181571365, val_loss:36.51573944091797
epoch:4, train_loss:37.21148258070872, val_loss:36.33281326293945
epoch:5, train_loss:37.226014154068544, val_loss:36.36334228515625
epoch:6, train_loss:37.246051863438105, val_loss:36.31577682495117
epoch:7, train_loss:37.19211706546922, val_loss:36.6376953125
epoch:8, train_loss:37.13130149940135, val_loss:36.327266693115234
epoch:9, train_loss:37.148124244176046, val_loss:36.40510559082031
Training completed in 193.1802475452423s
